## Load language model then loop through increasing number of training trials

In [ ]:
import sys
import os
sys.path.append('../../')
import lm_decoder
from utils.lmutils import cer_with_lm_decoder, build_lm_decoder


lmDir = '../../LanguageModels/Typing5gramPunc'

ngramDecoder = build_lm_decoder(
    lmDir,
    max_active=7000,
    min_active=200,
    beam=17,
    lattice_beam=8,
    acoustic_scale=0.23, #lower means weight LM more highly
    ctc_blank_skip_threshold=1.0,
    length_penalty=0.0,
    nbest=100)

In [ ]:
import os
import sys
sys.path.append('../../')
from pathlib import Path
import numpy as np
from T18_SessionArgs_numtrain import get_session_info
import utils.mat_to_tfrecord
from utils.lmutils import cer_with_lm_decoder
num_train_trials = [10,20,30,40,50,60,70,80,90,100]

session = 't18.2025.01.15' # 98 trials total
participant = 't18'
bin_compression_factor = 2
channels_to_exclude = list(range(0,0)) 
channels_to_zero = list(range(0,0)) 
session_path = str(Path('../../Data', participant, session, 'Typing'))
tfdata_path = str(Path(session_path, 'tfdata_20ms'))
trials_to_remove, block_nums, _ = get_session_info(session)
print(f'Sesison path: {session_path}')
print(f'tfdata path: {session_path}')

for num_train in num_train_trials:
    
    num_test_trials = 98 - num_train
    
    print('\n')
    
    args = {
        'session_mat_path': session_path,
        'block_nums': block_nums,
        'num_test_trials': num_test_trials,
        'trials_to_remove': trials_to_remove,
        'channels_to_exclude': channels_to_exclude,
        'channels_to_zero': channels_to_zero,
        'include_thresh_crossings': True,
        'include_spike_power': True,
        'spike_pow_max': 50000,
        'z_score_data': True,
        'global_std': True,
        'bin_compression_factor': bin_compression_factor,
        'save_path': tfdata_path,
    }
    
    utils.mat_to_tfrecord.main(args)


    from omegaconf import OmegaConf
    import sys
    sys.path.append('../../')
    from utils.brainToText_trainDecoder import brainToText_decoder
    
    args = OmegaConf.load('T18_trainArgs_numtrain.yaml')
    args['loadDir'] = 'null'
    args['outputDir'] = 'T18Models/' + str(num_train) + '-Train'
    decoder = brainToText_decoder(args)
    
    infOut, stats = decoder.train()


    from omegaconf import OmegaConf
    import sys
    sys.path.append('../../')
    from utils.brainToText_trainDecoder import brainToText_decoder
    
    args = OmegaConf.load('T18_trainArgs_numtrain.yaml')
    args['loadDir'] = 'T18Models/' + str(num_train) + '-Train'
    decoder = brainToText_decoder(args)

    out, out_by_day = decoder.inference()
    decoder_out = cer_with_lm_decoder(ngramDecoder, out, blankPenalty=1, rescore=False)
    np.save('T18Train' + str(num_train) + 'PostLMWERs.npy', decoder_out['all_wer'])